In [1]:
import os 
import json

In [2]:
def digest(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #df = pd.read_json(file, lines=True)
    return data


# Vehicle positions


In [3]:
entries = [digest(entry) for entry in os.scandir('../../data/') if entry.is_file()]

In [4]:
entries[0][2]['features'][4]['properties']


{'trip': {'cis_agency_name': 'ABOUT ME',
  'cis_id': '100166',
  'cis_number': 96,
  'cis_order': 5,
  'cis_parent_route_name': '166',
  'cis_real_agency_name': 'ABOUT ME',
  'cis_short_name': '166',
  'cis_vehicle_registration_number': 1935,
  'gtfs_route_id': 'L166',
  'gtfs_route_short_name': '166',
  'gtfs_trip_id': '166_292_181001',
  'id': '2019-08-12T19:09:00Z_100166_166_96',
  'start_cis_stop_id': 59501,
  'start_cis_stop_platform_code': 'B',
  'start_time': '21:09:00',
  'start_timestamp': '2019-08-12T19:09:00.000Z',
  'vehicle_type': 3,
  'wheelchair_accessible': True},
 'last_position': {'bearing': 75,
  'cis_last_stop_id': 59395,
  'cis_last_stop_sequence': 33,
  'delay': 269,
  'delay_stop_arrival': None,
  'delay_stop_departure': 284,
  'gtfs_last_stop_id': None,
  'gtfs_last_stop_sequence': None,
  'gtfs_next_stop_id': 'U63Z1P',
  'gtfs_next_stop_sequence': None,
  'gtfs_shape_dist_traveled': '16.8',
  'is_canceled': False,
  'lat': '50.15097',
  'lng': '14.51525',
  'or

In [32]:
import psycopg2

In [33]:
conn = psycopg2.connect("host=10.0.0.4 port=5432 dbname=pid-portal user=postgres password=passwd")

In [34]:
cur = conn.cursor()

In [35]:
cur.execute('SELECT * FROM "User"')

In [36]:
cur.fetchone()

(1, 'test', 'test')

In [21]:
i = 0
for line in entries[0]:
    for data in line['features']:
        cur.execute("""
                INSERT INTO "Vehicle_position" (vehicle_id, timestamp, bearing, calculated_delay, speed, longitude, latitude)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,
               (data['properties']['trip']['gtfs_route_id'],data['properties']['trip']['start_timestamp'], 
                data['properties']['last_position']['bearing'], data['properties']['last_position']['delay'],
                data['properties']['last_position']['speed'], data['properties']['last_position']['lng'], data['properties']['last_position']['lat']))
        i += 1

In [22]:
conn.commit()

# Stops

In [47]:
def get_bool_from_int_for_wheelchair(bool_val):
    if bool_val == 1:
        return "1"
    else:
        return "0"

In [37]:
def digest_stops(file):
    f = open(file, 'r')
    #data = [json.loads(line) for line in open(file, 'r')]
    data = json.load(f)
    return data

In [38]:
entries = [digest_stops(entry) for entry in os.scandir('/mnt/d/data_diplomka/stops') if entry.is_file()]

In [39]:
entries[0]['features'][5]['properties']

{'level_id': '',
 'location_type': 2,
 'parent_station': 'U1000S1',
 'platform_code': '',
 'stop_code': None,
 'stop_desc': None,
 'stop_id': 'U1000S1E3',
 'stop_lat': 50.12497,
 'stop_lon': 14.51587,
 'stop_name': 'Listova ul.',
 'stop_timezone': None,
 'stop_url': '',
 'wheelchair_boarding': 1,
 'zone_id': '',
 'create_batch_id': None,
 'created_at': '2019-09-26T03:00:21.913Z',
 'created_by': None,
 'update_batch_id': None,
 'updated_at': '2019-09-26T03:00:21.913Z',
 'updated_by': None}

In [51]:
for a in entries[0]['features']:
    i = 0
    cur.execute("""
    INSERT INTO public."Stop"(
	stop_id, name, lat, lng, zone_id, wheelchair_acc)
	VALUES (%s, %s, %s, %s, %s, %s);
    """,(a['properties']['stop_id'], a['properties']['stop_name'], a['properties']['stop_lat'], a['properties']['stop_lon'], a['properties']['zone_id'], 
        get_bool_from_int_for_wheelchair(a['properties']['wheelchair_boarding'])
        ))
    i += 1
print('Executed {} entries'.format(i))

Executed 1 entries


In [52]:
conn.commit()

In [42]:
eval(1)

TypeError: eval() arg 1 must be a string, bytes or code object